# Ag-Analytics® - Tillage Model API

The Ag-Analytics® Tillage Model API uses artificial intelligence algorithms to determine whether a field has been tilled at a certain date of the year. The Tillage Model API provides service by considering satellite imagery, field location, and time of the year.

The Tillage Model is a “backfilling model”, meaning that it can determine whether a field was tilled in the past, rather than provide prediction as to whether a field will be tilled. The model uses a seven-week window of landsat imagery, so as long as the date requested is more than three weeks prior to the current date (i.e., if today is Monday of the 20th week of the year, the latest request that can be made is for the 16th week.).


In [9]:
import requests
import json
import time
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


### Request Parameters Details:
####Request URL: https://ag-analytics.azure-api.net/tillage-model/[?f][&geometry]

1) ***ModelType*** (Text, optional): Determines the type of model to be used for prediction. Can be one of the following:
- "RF": Random forest classifier
- "NN": Neural network
- "SVM": Support vector machine

2) ***SHAPE*** (geometry, file/text, required): The boundary of the field of interest.

3) ***TillageStartDate*** (Date, mm-dd-yyyy, required): The date of interest for predicting tillage.


### Request Parameters

The Tillage API service takes the request in Json format. Below are examples formatted in Json.

In [156]:
values = {
"ModelType": "SVM",
"SHAPE": "{\"type\":\"Feature\",\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-89.199484,40.972729],[-89.199773,40.97258],[-89.200135,40.972415],[-89.20034,40.972318],[-89.200445,40.972177],[-89.200439,40.972001],[-89.200404,40.971815],[-89.200245,40.971599],[-89.20004,40.971397],[-89.199869,40.971233],[-89.199865,40.971097],[-89.199952,40.970952],[-89.200264,40.97078],[-89.200517,40.970664],[-89.200903,40.970471],[-89.201168,40.970345],[-89.201324,40.970277],[-89.201407,40.970174],[-89.201428,40.970042],[-89.20271,40.970005],[-89.202738,40.970421],[-89.202844,40.970431],[-89.202851,40.970648],[-89.203123,40.970666],[-89.203216,40.973626],[-89.20332,40.973635],[-89.203281,40.972154],[-89.203277,40.972049],[-89.203227,40.970607],[-89.204645,40.97055],[-89.204639,40.970427],[-89.205456,40.970446],[-89.205638,40.970467],[-89.206002,40.970527],[-89.206306,40.97059],[-89.206516,40.970642],[-89.206711,40.97061],[-89.20688,40.970542],[-89.207086,40.970492],[-89.207267,40.970414],[-89.207449,40.970364],[-89.207667,40.970286],[-89.207849,40.970255],[-89.208057,40.970251],[-89.208287,40.970328],[-89.208494,40.970369],[-89.208672,40.970421],[-89.208866,40.970506],[-89.208972,40.970511],[-89.209009,40.970595],[-89.20893,40.970671],[-89.208736,40.970787],[-89.208535,40.970909],[-89.208325,40.971052],[-89.207907,40.971306],[-89.207633,40.971478],[-89.207313,40.971574],[-89.207065,40.971645],[-89.206566,40.971699],[-89.206246,40.971784],[-89.205998,40.971878],[-89.205548,40.972042],[-89.205013,40.97232],[-89.20468,40.972494],[-89.204246,40.972725],[-89.203988,40.972931],[-89.203819,40.973168],[-89.203666,40.973428],[-89.203616,40.973685],[-89.203552,40.973966],[-89.203548,40.9743],[-89.203411,40.974615],[-89.203284,40.974906],[-89.202723,40.975587],[-89.20283,40.975719],[-89.203383,40.975106],[-89.203522,40.974847],[-89.203658,40.974521],[-89.203723,40.974241],[-89.20381,40.97376],[-89.203891,40.973546],[-89.20407,40.973197],[-89.204197,40.973016],[-89.204369,40.972868],[-89.204686,40.972672],[-89.205018,40.972499],[-89.205351,40.972314],[-89.205742,40.972139],[-89.206047,40.971999],[-89.206367,40.971904],[-89.206907,40.971771],[-89.207303,40.971719],[-89.207551,40.971658],[-89.207846,40.971535],[-89.207938,40.971481],[-89.208059,40.971448],[-89.208267,40.971295],[-89.208534,40.971115],[-89.209089,40.970762],[-89.209108,40.971493],[-89.209143,40.972829],[-89.209176,40.974108],[-89.209236,40.977186],[-89.20442,40.977285],[-89.199613,40.977383],[-89.199533,40.974593],[-89.199484,40.972729]]]},\"properties\":{\"OBJECTID\":5102679,\"CALCACRES\":145.08999634,\"CALCACRES2\":null},\"id\":5102679}",
"ScalarVariables": {
"TillageStartDate": "04-23-2018"
}
}

# Basic header pattern.
headers={'Content-Type':'application/json'}

# Header for using a subscription key.
# headers={'Content-Type':'application/x-www-form-urlencoded','Ocp-Apim-Subscription-Key': '••••••••••••••••••••••••••••••••'}

### API Fuction

In [160]:
def tillage_forecast(values,headers):
    try:
        url = "https://ag-analytics.azure-api.net/tillage-model/"
        url = "http://127.0.0.1:5000/agflask/api/v1.0/Tillage_Model_Forecast"
     
        response = requests.post(url, json=values, headers=headers).json()
        
        print(response)

        return response
    
    except Exception as e:
        print(e)
        raise e

### Calling API Function and Displaying Response

In [161]:
tillage_response=tillage_forecast(values,headers)

{'feature_averages': [{'Key': 'x', 'Value': -89.20441134369396}, {'Key': 'y', 'Value': 40.973876813508674}], 'raster_filename': 'result_tillageraster_20200626_204635_7403.tif', 'rasterinfo': [{'attributes': {'CellSize': [0.0001, -0.0001], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-89.209236, 40.969983000000006, -89.19953600000001, 40.977383', 'Legend': [{'Area': 3.262830828783441, 'AreaPercent': '1.02 %', 'AreaUnit': 'ac', 'Count': 68, 'CountAllPixels': 6692, 'Till': 'No', 'Value': -1, 'Variety': 'Variety -1.0', 'color': '#4bbe84'}, {'Area': 317.8381089685517, 'AreaPercent': '98.98 %', 'AreaUnit': 'ac', 'Count': 6624, 'CountAllPixels': 6692, 'Till': 'Yes', 'Value': 1, 'Variety': 'Variety 1.0', 'color': '#7f0f41'}], 'pngb64': 'data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAGEAAABKCAYAAACrbT

### Displaying Output Image

In [162]:
# Image legend.
df=defaultdict(list)
for resp in tillage_response["rasterinfo"][0]['attributes']['Legend']:
    df['Till'].append(resp['Till'])
    df['Till Value'].append(resp['Value'])
    df['Area (ac)'].append(round(resp['Area']))
    df['Color'].append(resp['color'])
    

# Create a Data Frame from the dictionary and display.
tillagedf=pd.DataFrame.from_dict(df)
tillagedf = tillagedf.style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])
tillagedf

In [163]:
# Display image of field.
tillage_image = tillage_response["rasterinfo"][0]['attributes']["pngb64"]
Image(url = tillage_image, width = 400, height = 200)

## GET Request to Retrieve GeoTIFF

In [164]:
# Specify local path where file will be downloaded.
local_path = r"C:\Users\Alex\Documents" # E.g., r"C:\Users\John_Doe\Documents\rasters"
# Create GET request payload.
values = {'filename': tillage_response['raster_filename']}
print(values)

{'filename': 'result_tillageraster_20200626_204635_7403.tif'}


In [167]:
def tillage_get(values, local_path):
    try:
        url = 'https://ag-analytics.azure-api.net/tillage-model/'
        
        print('values', values)

        response = requests.get(url, params=values)
        
        download_path = os.path.join
        open(download_path, 'wb').write(response.content)

        print(response.url)

        print(response)
    
    except Exception as e:
        print(e)
        raise e

In [168]:
tillage_get(values, local_path)

values {'filename': 'result_tillageraster_20200626_204635_7403.tif'}
name 'download_path' is not defined


NameError: name 'download_path' is not defined